# Dependencies

# Functions

# Paths

In [1]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

In [9]:
hla <- xlsx::read.xlsx(file = paste0(datapath, "/HLA/UHN-20230127-B257-HLA.xlsx"), sheetIndex = 1)

In [12]:
head(hla,20)

,Sample.ID,Locus,Allele.1,Allele.2,Comments,Diploid.Ambiguities,Allele.1.Ambiguities,Allele.2.Ambiguities
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,315-05,A,A*02:01:01,A*02:05:01,,,,
2,315-05,B,B*35:01:01,B*49:01:01,,,,
3,315-05,C,C*04:01:01,C*07:01:01,,,,
4,315-05,DPA1,DPA1*01:03:01,DPA1*02:02:02,,,,
5,315-05,DPB1,DPB1*01:01:01,DPB1*04:02:01,,,,
6,315-05,DQA1,DQA1*01:02:01,DQA1*05:05:01,,,,
7,315-05,DQB1,DQB1*03:19:01,DQB1*06:04:01,,,,
8,315-05,DRB1,DRB1*13:02:01,DRB1*13:04,,,,
9,315-05,DRB345,DRB3*02:02:01,DRB3*03:01:01,,,,
